# Capstone project-Battle of quarters

## Business problems section

### Background

According to Bloomberg News, the London housing market is in a rut. It is currently facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that the cost of housing in the UK could fall by as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, the four overlooked cracks suggest that the London market may be in worse shape than many assume: hidden price falls, record low sales, an Exodus of Housebuilders and tax hikes targeting overseas home buyers in England and Wales.

### business problem

In this case, there is an urgent need to adopt machine learning tools to help London home buyers make wise and effective decisions. As a result, the business problem we are currently facing is the following: how could we support homebuyers clients in purchasing suitable properties in London in this uncertain economic and financial scenario?
To solve this business problem, we are going to bring together London boroughs to recommend locations and the current average property price where homebuyers can make a property investment. We will recommend favorable locations according to the amenities and basic amenities surrounding places such as elementary schools, high schools, hospitals, and grocery stores.

### Data section

Data on London properties and the relative price paid were extracted from the HM land registry (http://landregistry.data.gov.uk/). the following fields contain address data included in the paid price data: postal code; name of the main address object paon. This is usually a house number or name; caon is a secondary address object. If there is an extension, for example, the building is divided into apartments, then there will be a district; street; locality; city/city; district; district.

To explore and navigate recommended locations in various locations depending on the availability of amenities and basic amenities, we will access data through the Foursquare API and organize it as a data frame for visualization. By combining data on London properties and the relative price paid from the HM land registry, as well as data on amenities and major properties surrounding such properties from the Foursquare API, we can recommend profitable real estate investments.

## Methodology section
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:
1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

### 1. Collect Inspection Data
After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [4]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [6]:
df_ppd.head(5)

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [7]:
df_ppd.shape

(1029228, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.

### 3. Data preparation and preprocessing
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:
* Rename the column names
* Format the date column
* Sort data by date of sale
* Select data only for the city of London
* Make a list of street names in London
* Calculate the street-wise average price of the property
* Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
* Join the data to find the coordinates of locations which fit into client's budget
* Plot recommended locations on London map along with current market prices

In [8]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [9]:
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [10]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

streets = df_ppd_london['Street'].unique().tolist()

In [11]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

df_grp_price.columns = ['Street', 'Avg_Price']

In [12]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [13]:
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
855,BARONSMEAD ROAD,2375000.0
...,...,...
13718,WILFRED STREET,2410538.5
13744,WILLOW BRIDGE ROAD,2425000.0
13764,WILSON STREET,2257500.0
13792,WINCHENDON ROAD,2350000.0


In [14]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from sklearn.cluster import KMeans

In [15]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [16]:
geolocator = Nominatim()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [17]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(51.4803164, -0.1668011)"
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2400000.0,"(53.7486539, -0.3266704)"
855,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"
...,...,...,...
13718,WILFRED STREET,2410538.5,"(51.442178, 0.372743)"
13744,WILLOW BRIDGE ROAD,2425000.0,"(53.6408743, -1.2004617)"
13764,WILSON STREET,2257500.0,"(42.286454, -71.4009814)"
13792,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)"


In [19]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [20]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457
...,...,...,...,...,...
13718,WILFRED STREET,2410538.5,"(51.442178, 0.372743)",51.442178,0.372743
13744,WILLOW BRIDGE ROAD,2425000.0,"(53.6408743, -1.2004617)",53.640874,-1.200462
13764,WILSON STREET,2257500.0,"(42.286454, -71.4009814)",42.286454,-71.400981
13792,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)",42.645331,-71.954714


In [21]:
df = df_affordable.drop(columns=['city_coord'])
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457
...,...,...,...,...
13718,WILFRED STREET,2410538.5,51.442178,0.372743
13744,WILLOW BRIDGE ROAD,2425000.0,53.640874,-1.200462
13764,WILSON STREET,2257500.0,42.286454,-71.400981
13792,WINCHENDON ROAD,2350000.0,42.645331,-71.954714


In [22]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [23]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [24]:
CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### 4. Modeling
After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [27]:
location_venues


,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
...,...,...,...,...,...,...,...
4286,WILSON STREET,42.286454,-71.400981,Captain Tom's Hilltop Tap,42.286893,-71.396080,Apres Ski Bar
4287,WINGATE ROAD,51.092557,1.179455,Currys PC World,51.093848,1.179104,Electronics Store
4288,WINGATE ROAD,51.092557,1.179455,The Hungry Horse,51.089400,1.180624,Gastropub
4289,WINGATE ROAD,51.092557,1.179455,Nisa Local,51.095157,1.184555,Supermarket


In [28]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,27,27,27,27,27,27
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,44,44,44,44,44,44
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,13,13,13,13,13,13
...,...,...,...,...,...,...
WESTMORELAND PLACE,18,18,18,18,18,18
WHITFIELD STREET,5,5,5,5,5,5
WILFRED STREET,25,25,25,25,25,25


In [29]:
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))


There are 337 uniques categories.


In [30]:
location_venues.shape

(4291, 7)

In [31]:
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

venues_onehot['Street'] = location_venues['Street'] 

fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Apres Ski Bar,Arcade,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Apres Ski Bar,Arcade,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,ANHALT ROAD,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,ANSDELL TERRACE,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,APPLEGARTH ROAD,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,BARONSMEAD ROAD,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,WESTMORELAND PLACE,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
145,WHITFIELD STREET,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
146,WILFRED STREET,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.00,0.04,0.0,0.0,0.04,0.0,0.0,0.0,0.0
147,WILSON STREET,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [33]:
london_grouped.shape

(149, 338)

In [34]:
num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
         venue  freq
0         Café  0.22
1   Restaurant  0.07
2          Bar  0.07
3          Pub  0.07
4  Coffee Shop  0.07


----ANHALT ROAD----
               venue  freq
0                Pub  0.27
1      Grocery Store  0.13
2       Cocktail Bar  0.07
3  French Restaurant  0.07
4             Garden  0.07


----ANSDELL TERRACE----
                venue  freq
0           Juice Bar  0.07
1  Italian Restaurant  0.07
2          Restaurant  0.07
3      Clothing Store  0.07
4               Hotel  0.07


----APPLEGARTH ROAD----
       venue  freq
0        Pub  0.25
1  Nightclub  0.25
2        Bar  0.25
3     Casino  0.25
4        ATM  0.00


----BARONSMEAD ROAD----
                 venue  freq
0    Food & Drink Shop  0.15
1  Indie Movie Theater  0.08
2          Coffee Shop  0.08
3                  Pub  0.08
4        Movie Theater  0.08


----BEAUCLERC ROAD----
             venue  freq
0  Harbor / Marina   0.4
1        Speakeasy   0.2
2  Automotive Shop   0.2
3      Pi

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


In [37]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [38]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Coffee Shop,Bar,Indian Restaurant,Pub,Restaurant,Seafood Restaurant,Brewery,Park,Beer Garden
1,ANHALT ROAD,Pub,Grocery Store,Garden,English Restaurant,Plaza,French Restaurant,Gym / Fitness Center,Cocktail Bar,Japanese Restaurant,Diner
2,ANSDELL TERRACE,Juice Bar,Hotel,Italian Restaurant,Restaurant,Clothing Store,Indian Restaurant,Chinese Restaurant,Pub,Bakery,Furniture / Home Store
3,APPLEGARTH ROAD,Pub,Bar,Nightclub,Casino,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
4,BARONSMEAD ROAD,Food & Drink Shop,Movie Theater,Breakfast Spot,Pub,Park,Indie Movie Theater,Pizza Place,Coffee Shop,Nature Preserve,Restaurant


In [39]:
venues_sorted.shape

(149, 11)

In [40]:
london_grouped.shape

(149, 338)

In [41]:
london_grouped=df

In [42]:
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

kmeans.labels_[0:50]

array([1, 4, 2, 4, 3, 1, 3, 2, 2, 1, 4, 4, 1, 1, 3, 3, 1, 4, 2, 1, 0, 0,
       4, 1, 1, 2, 4, 0, 1, 2, 4, 3, 4, 3, 3, 0, 4, 4, 3, 2, 1, 3, 0, 2,
       0, 2, 2, 0, 2, 2], dtype=int32)

In [43]:
london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [44]:
london_grouped_clustering.shape

(162, 4)

In [45]:
df.shape

(162, 4)

In [46]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [47]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [48]:
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30)

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Coffee Shop,Bar,Indian Restaurant,Pub,Restaurant,Seafood Restaurant,Brewery,Park,Beer Garden
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,4,Pub,Grocery Store,Garden,English Restaurant,Plaza,French Restaurant,Gym / Fitness Center,Cocktail Bar,Japanese Restaurant,Diner
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,2,Juice Bar,Hotel,Italian Restaurant,Restaurant,Clothing Store,Indian Restaurant,Chinese Restaurant,Pub,Bakery,Furniture / Home Store
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,4,Pub,Bar,Nightclub,Casino,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,3,Food & Drink Shop,Movie Theater,Breakfast Spot,Pub,Park,Indie Movie Theater,Pizza Place,Coffee Shop,Nature Preserve,Restaurant
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Harbor / Marina,Speakeasy,Automotive Shop,Pizza Place,Zoo,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796,3,Pool,Playground,Athletics & Sports,Zoo,Falafel Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,2,Pizza Place,Café,Italian Restaurant,Restaurant,Gastropub,Bar,Movie Theater,Garden,Hotel,Bakery
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,2,Pub,Brewery,French Restaurant,Lake,Coffee Shop,Chinese Restaurant,Train Station,Bakery,Factory,Electronics Store
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,Home Service,Zoo,Farm,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory


In [49]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2.300000e+06,Farm,Zoo,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
2405,2.286679e+06,Café,Pub,Italian Restaurant,Bar,Coffee Shop,French Restaurant,Park,Restaurant,Bakery,Breakfast Spot
2685,2.287500e+06,Pub,Art Museum,Brewery,Gym / Fitness Center,Gift Shop,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
3376,2.298000e+06,Hotel,Zoo,Farm,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
4284,2.265000e+06,Pub,Zoo,Farm,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory


In [51]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Coffee Shop,Bar,Indian Restaurant,Pub,Restaurant,Seafood Restaurant,Brewery,Park,Beer Garden
981,2480000.0,Harbor / Marina,Speakeasy,Automotive Shop,Pizza Place,Zoo,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
1553,2456875.0,Home Service,Zoo,Farm,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1914,2445000.0,Grocery Store,Gym,Bar,Athletics & Sports,Dance Studio,Falafel Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1980,2492500.0,Supermarket,English Restaurant,Pub,Coffee Shop,Gym,Café,Hardware Store,Park,Dry Cleaner,Rental Car Location


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Juice Bar,Hotel,Italian Restaurant,Restaurant,Clothing Store,Indian Restaurant,Chinese Restaurant,Pub,Bakery,Furniture / Home Store
1215,2208500.0,Pizza Place,Café,Italian Restaurant,Restaurant,Gastropub,Bar,Movie Theater,Garden,Hotel,Bakery
1253,2217000.0,Pub,Brewery,French Restaurant,Lake,Coffee Shop,Chinese Restaurant,Train Station,Bakery,Factory,Electronics Store
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2637,2250000.0,Hotel,Pharmacy,Bakery,Bookstore,Pizza Place,Coffee Shop,Italian Restaurant,Grocery Store,Gift Shop,Supermarket


In [53]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
855,2375000.0,Food & Drink Shop,Movie Theater,Breakfast Spot,Pub,Park,Indie Movie Theater,Pizza Place,Coffee Shop,Nature Preserve,Restaurant
1102,2340000.0,Pool,Playground,Athletics & Sports,Zoo,Falafel Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
2068,2375000.0,Pub,Park,Pizza Place,Hotel,Yoga Studio,Indian Restaurant,Grocery Store,Ice Cream Shop,Bus Stop,Gastropub
2129,2379652.7,Pub,Grocery Store,Pizza Place,Hotel,Coffee Shop,Park,Yoga Studio,Indian Restaurant,Bakery,Burger Joint
2943,2367500.0,Hotel,Pub,Garden,Coffee Shop,Italian Restaurant,Café,Mediterranean Restaurant,Chinese Restaurant,Bar,Middle Eastern Restaurant


In [54]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,Garden,English Restaurant,Plaza,French Restaurant,Gym / Fitness Center,Cocktail Bar,Japanese Restaurant,Diner
422,2400000.0,Pub,Bar,Nightclub,Casino,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1632,2397132.0,Convenience Store,Grocery Store,Coffee Shop,Italian Restaurant,Sushi Restaurant,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1797,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2158,2425000.0,Clothing Store,Electronics Store,Women's Store,Convenience Store,Basketball Court,Stationery Store,American Restaurant,Gym,Department Store,Chinese Restaurant


# Results and discussion section

First of all, despite the fact that the London housing market may be in a rut, it still remains "forever green" for business.

We can discuss our results from two main points of view.

First, we can research them according to London boroughs / districts. It is interesting to note that while West London (Notting hill, Kensington, Chelsea, Marylebone) and North West London (Hampstead) can be considered very profitable places to buy property according to the amenities and basic amenities surrounding places such as primary schools, secondary schools, hospitals and grocery stores, South West London (Wandsworth, Balham) and North West London (Islington) are emerging as the next future elite locations with a wide range of amenities and amenities. Accordingly, it would be possible to focus on low property prices in these areas of London to make a business case.

Second, we can analyze our results according to the five clusters we have created. Despite the fact that all clusters could praise the optimal set of facilities and amenities, we found two main patterns. The first model we are talking about, i.e. clusters 0, 2 and 4, can be aimed at home buyers who tend to live in "green" areas with parks and embankments. Instead, the second model we are talking about, i.e. clusters 1 and 3, can be aimed at people who like pubs, theaters and football.

# Conclusion

To sum up, according to Bloomberg News, the London housing market is in a rut. It is currently facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that the cost of housing in the UK could fall by as much as 30 percent in the event of a disorderly exit from the European Union. In this case, there is an urgent need to adopt machine learning tools to help London home buyers make wise and effective decisions. As a result, the business challenge we set for ourselves was: how could we support homebuyers clients in purchasing suitable properties in London in this uncertain economic and financial scenario?

To solve this business problem, we have grouped London boroughs to recommend locations and the current average property price where homebuyers can make property investments. We recommended favorable locations according to the amenities and basic amenities surrounding places such as elementary schools, high schools, hospitals, and grocery stores.

First, we collected data on London real estate, and the relative price data was extracted from the HM land registry (http://landregistry.data.gov.uk in addition, to explore and navigate the recommended locations in various locations depending on the availability of amenities and major facilities, we accessed the data via the Foursquare API and arranged it as a data frame for visualization. By combining data on London properties and the relative price paid from the HM land registry, as well as data on amenities and major properties surrounding such properties from the Foursquare API, we were able to recommend profitable real estate investments.

Second, the methodology section consisted of four stages: 1. collecting inspection data; 2. Researching and understanding data; 3.Preparing and pre-processing data; 4.Modeling. In particular, in the modeling section, we used the K-means clustering method, because it is fast and efficient in terms of computational costs, very flexible for accounting for mutations in the London property market, and accurate.

Finally, we concluded that while the London housing market may be in a rut, it is still "forever green" for business. We discussed our results from two main points of view. First, we looked at them according to London boroughs / districts. although West London (Notting hill, Kensington, Chelsea, Marylebone) and North-West London (Hampstead) can be considered very profitable places to buy property according to the amenities and basic amenities surrounding such places, i.e. primary schools, secondary schools, hospitals and grocery stores, South West London (Wandsworth, Balham) and North West London (Islington) are becoming the next future luxury properties with a wide range of amenities and amenities. Accordingly, it would be possible to focus on low property prices in these areas of London to make a business case. Second, we analyzed our results for the five clusters we created. While clusters 0, 2, and 4 may target home buyers who tend to live in "green" areas with parks, embankments, clusters 1 and 3 may target people who like pubs, theaters, and soccer.